## The Back Story...

Shakespeare ressurrected in the 21st century and needs to keep up with modern technology. On top of that, he has to find supplementary source of income since his playwrighting skills are no longer relevant in this day and age and in the midst of the cost of living crisis. He eventually found a job as a customer support specialist, but he still has difficulty understanding the modern English lingo. Equipped with some new tech skills he learned on the internet, Shakespeare now wants to explore Large Language Models to help him translate customer prompts in modern English into the English that he once knew...

### A sample customer prompt

In [37]:
customer_prompt = '''
Hey, I accidentally spilled oat milk on my toaster \
while making breakfast this morning. I unplugged it \
and waited for a few minutes before turning it back \
on, but now it's just making a weird buzzing noise \
and the indicator light doesn't switch on. Help me!
'''

In [38]:
style = 'like the plays of Shakespeare'

prompt = f'''Translate the text that is delimited by \
triple backticks into a style that is {style}.
text: ```{customer_prompt}```
'''

## Part 1: Direct API calls to OpenAI

In [36]:
import openai
import os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [39]:
def chat_with_openai(prompt, model='gpt-3.5-turbo'):
    message = {
        'role': 'user',
        'content': prompt,
    }
    # A lower temperature will make responses more deterministic
    response = openai.ChatCompletion.create(model=model,
                                            messages=[message],
                                            temperature=0)
    tokens = response['usage']['total_tokens']
    response = response['choices'][0]['message']['content']
    
    return response, tokens

In [40]:
response, tokens_used = chat_with_openai(prompt)

In [41]:
response

"Hark! 'Tis a tale of woe that I doth recount,\nForsooth, this morn whilst breaking fast, I erred.\nOat milk, by chance, did spill upon my toaster,\nAn accident, yet dire consequences ensued.\n\nSwiftly, I did unplug the device with care,\nAnd patiently, I waited for minutes three,\nEre I dared to rekindle its electric flame.\nAlas! A strange buzzing sound doth now persist,\nAnd the light, once bright, doth refuse to shine.\n\nPray, I beseech thee, lend me thy aid!\nForsooth, I am in need of thy wise counsel,\nTo mend this toaster, plagued by misfortune."

In [42]:
tokens_used

235

## Part 2: Using LangChain

In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [43]:
prompt_template = ChatPromptTemplate.from_template(prompt)

In [44]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="Translate the text that is delimited by triple backticks into a style that is like the plays of Shakespeare.\ntext: ```\nHey, I accidentally spilled oat milk on my toaster while making breakfast this morning. I unplugged it and waited for a few minutes before turning it back on, but now it's just making a weird buzzing noise and the indicator light doesn't switch on. Help me!\n```\n", template_format='f-string', validate_template=True)

In [50]:
chat_messages = prompt_template.format_messages(
                        style=style,
                        text=customer_prompt)

In [51]:
chat_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is like the plays of Shakespeare.\ntext: ```\nHey, I accidentally spilled oat milk on my toaster while making breakfast this morning. I unplugged it and waited for a few minutes before turning it back on, but now it's just making a weird buzzing noise and the indicator light doesn't switch on. Help me!\n```\n", additional_kwargs={}, example=False)

In [54]:
chat = ChatOpenAI(temperature=0.0)
response = chat(chat_messages)

In [55]:
print(response.content)

Hark! 'Tis a tale of woe that I doth recount,
Forsooth, this morn whilst breaking fast in haste,
Mine hand didst falter, and oat milk didst spill,
Upon mine toaster, causing great distress.

Swiftly, I didst unplug the wretched device,
And bide my time, a few minutes didst pass,
Ere I dared to rekindle its fiery glow,
Yet, alas! A strange buzzing doth it emit,
And the light, once bright, doth refuse to shine.

Oh, I beseech thee, lend me thine aid,
Forsooth, what remedy canst thou proffer?
Pray, help me mend this toaster's sorry state,
That it may once again serve me with grace.


In [56]:
response

AIMessage(content="Hark! 'Tis a tale of woe that I doth recount,\nForsooth, this morn whilst breaking fast in haste,\nMine hand didst falter, and oat milk didst spill,\nUpon mine toaster, causing great distress.\n\nSwiftly, I didst unplug the wretched device,\nAnd bide my time, a few minutes didst pass,\nEre I dared to rekindle its fiery glow,\nYet, alas! A strange buzzing doth it emit,\nAnd the light, once bright, doth refuse to shine.\n\nOh, I beseech thee, lend me thine aid,\nForsooth, what remedy canst thou proffer?\nPray, help me mend this toaster's sorry state,\nThat it may once again serve me with grace.", additional_kwargs={}, example=False)